In [4]:
# !pip install -qU pip wheel
# !pip install -q -r ../requirements.txt
import sys
sys.path.append('..')
from ballitoreproject import *
from ballitoreproject.networks import *

In [7]:
G=get_family_tree_network()

plot_network(G, edge_color='relationship')

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'content': …

In [15]:

def get_correspondence_network(df=None, incl_family_tree=True, min_num_letters=10):
    G = nx.DiGraph() if not incl_family_tree else get_family_tree_network()
    dfx = (
        get_data()
        .query('sender!="" & recipient!=""')
        .groupby(["sender", "recipient"])[["num_letters", "num_words"]]
        .sum()
        .reset_index()
        .query(f"num_letters>={min_num_letters}")
        if df is None
        else dfx
    )
    for i, row in dfx.iterrows():
        if G.has_edge(row.sender, row.recipient):
            G.edges[(row.sender, row.recipient)]["num_letters"] = row.num_letters
            G.edges[(row.sender, row.recipient)]["num_words"] = row.num_words
        # elif G.has_node(row.sender) or G.has_node(row.recipient):
        else:
            G.add_edge(
                row.sender,
                row.recipient,
                relationship="wrote",
                num_letters=row.num_letters,
                num_words=row.num_words,
            )
    
    for n,d in G.nodes(data=True):
        if not 'is_family' in d:
            G.nodes[n]['is_family']=False

    return G



In [16]:
G=get_correspondence_network(min_num_letters=10)
G.order(), G.size()

(55, 63)

In [17]:
plot_network(G, 
    edge_width='num_letters',
    node_color='is_family',
    edge_color='relationship',
    # node_colormap='Pastel2'
)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'content': …